In [1]:
# Code of movie recommendations system and it`s reviews and all the details of the movie`s 

In [2]:
from nltk.corpus import movie_reviews
movie_reviews.categories()
movie_reviews.fileids()
len(movie_reviews.fileids())
print(movie_reviews.words(movie_reviews.fileids()[5]))

['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]


In [3]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid), category))
documents[0:5]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')]

In [4]:
import random 
random.shuffle(documents)
documents[0:5]

[(['i', 'still', 'can', "'", 't', 'figure', 'out', 'why', ...], 'neg'),
 (['casting', '"', 'doogie', 'howser', '"', 'star', ...], 'pos'),
 (['every', 'once', 'in', 'a', 'while', ',', 'when', ...], 'pos'),
 (['i', "'", 've', 'noticed', 'something', 'lately', ...], 'pos'),
 (['"', 'american', 'pie', '"', 'alums', 'jason', ...], 'neg')]

In [5]:
from nltk.corpus import wordnet
def get_simple_pos(tag): 
    if tag.starswith('J'): 
        return wordnet.ADJ 
    elif tag.startswith('V'): 
        return wordnet.VERB
    elif tag.startswith('N'): 
        return wordnet.NOUN
    elif tag.startswith('R'): 
        return wordnet.ADV 
    else: 
        return wordnet.NOUN

In [6]:
from nltk import pos_tag 
w = "better" 
pos_tag([w])

[('better', 'RBR')]

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
import string

# Set stopwords and punctuation
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)  # Corrected attribute
stops.update(punctuations)

lemmatizer = WordNetLemmatizer()

# Function to map POS tags to simpler tags for lemmatizer
def get_simple_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun

# Function to clean review text
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:  # Check if word is not in stopwords or punctuation
            pos = pos_tag([w])  # Correct usage of pos_tag
            clean_word = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

# Example document structure
# Make sure `documents` is tokenized properly
documents = [
    (["This", "is", "a", "sample", "document"], "category1"),
    (["Another", "example", "text"], "category2"),
]

# Clean reviews
documents = [(clean_review(document), category) for document, category in documents]

print(documents)

training_documents = documents[0:1500]
testing_documents = documents[1500:]

a = [1, 2]
b = [3, 4]
a += b 
a
all_words = []
for doc in training_documents: 
    all_words += doc[0]

print(all_words)

[(['sample', 'document'], 'category1'), (['another', 'example', 'text'], 'category2')]
['sample', 'document', 'another', 'example', 'text']


In [8]:
import nltk 
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]
features
documents[0]

(['sample', 'document'], 'category1')

In [9]:
def get_feature_dict(words): 
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set 
    return current_features 


In [10]:
output = get_feature_dict(training_documents[0][0])
output
training_documents[0][0]

['sample', 'document']

In [11]:
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
test_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
training_data[0]

({'sample': True,
  'document': True,
  'another': False,
  'example': False,
  'text': False},
 'category1')

In [12]:
from nltk import NaiveBayesClassifier 
classifier = NaiveBayesClassifier.train(training_data)
print(classifier)
nltk.classify.accuracy(classifier, test_data)
classifier.show_most_informative_features(15)



Most Informative Features


In [13]:
from sklearn.svm import SVC 
from nltk.classify.scikitlearn import SklearnClassifier  

svc = SVC()
classifier_sklearn = SklearnClassifier(svc)
classifier_sklearn.train(training_data)

nltk.classify.accuracy(classifier_sklearn, training_data)
nltk.classify.accuracy(classifier_sklearn, test_data)

1.0

In [14]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier()
classifier_sklearn = SklearnClassifier(rfc)
classifier_sklearn.train(training_data)
nltk.classify.accuracy(classifier_sklearn, test_data)
nltk.classify.accuracy 

<function nltk.classify.util.accuracy(classifier, gold)>

In [15]:
from sklearn.feature_extraction.text import CountVectorizer 
# Define the training set
train_set = ["The sky is blue", "The sun is bright"]
# Initialize CountVectorizer with a maximum of 3 features
count_vec = CountVectorizer(max_features=3)
# Fit and transform the training set
a = count_vec.fit_transform(train_set)
# Convert the sparse matrix to a dense representation
dense_matrix = a.todense()
# Get feature names
feature_names = count_vec.get_feature_names_out()
# Output results
print("Dense Matrix:\n", dense_matrix)
print("Feature Names:", feature_names)


Dense Matrix:
 [[1 1 1]
 [0 1 1]]
Feature Names: ['blue' 'is' 'the']


In [16]:
documents[0]
categories = [category for document, category in documents]
text_documents = [" ".join(document) for document, category in documents]
text_documents[0]

'sample document'

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_documents, categories)
count_vec = CountVectorizer(max_features = 3)
a = count_vec.fit_transform(x_train)
a.todense()
count_vec.get_feature_names_out()

array(['document', 'sample'], dtype=object)

In [18]:
x_test_features = count_vec.transform(x_test)
x_test_features 

<1x2 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>